# NLP Homework 3

## 作業3-1和處理資料
第一個作業基本就正常按照投影片做，用 Word2Vec 這個函數生成 model 。後來根據kaggle上面同學分享的經驗改進了一下：用pkl存儲所有文本。

## 作業3-2
作業 3-2，一開始思路比較混亂，甚至沒有考慮到要生成文章對應類別list，也就是 label_list。開始我是對每篇文章的每個切割後字詞的 word_embedding 加總 **取平均後** 去訓練，但是這樣預測出來結果都是一個分類，明顯錯了。後來請教助教的時候發現應該是壓縮取平均的過程中 **把每篇文章的差異性消除了**，所以才會導致分成一個類別。

正確的概念上的理解應該是把每一篇文章作爲一筆輸入資料 ，把一個 10 唯度的類別向量作爲一個對應的 label 。這樣訓練集有 9000 筆資料，測試結果同理有 1000 筆資料。而每筆資料中的文章，應該是這篇文章裏每個切割後的字詞的 word embedding 向量。

### 在實作過程中容易走彎路和容易混淆的地方
1. embedding_layer 必須作爲第一層輸入，它的內容是字典裏所有的 token 對應的 vector 。這個字典裏所有的 token 一般來說是不重復的，是我們能給出的訓練集裏出現過的字詞。本次作業中，我設定了每個字詞的 word_embedding 是 250 維的 vector ，所以這個 embedding_layer 的shape 應該是 250 X 232236 。
2. 喂給神經網絡模型的 fit 函數的應該是文章序列以及對應的label，維度都是 9000 X 1 和 9000 X 10 。文章序列裏每篇文章的格式應該是數字，準確的說是每個字詞在上述字典裏的 ID 值。**在訓練的時候，keras 會自動在第一層 embedding_layer 中把這個值轉換成 word_embedding 的vector 格式** ，所以不用我們自己轉換成 word_embedding 後再輸入給 fit 函數。
3. 由於 keras 和電腦本身更加適應統一長度的輸入，所以實作中需要把長度不一的文章填補一些 data ，以形成統一長度的格式，本次作業中統一填補成 3971 長度。因爲最長的文章就是只有 3971 個字詞。

### 神經網絡模型
第一層是 embedding_layer ，輸入的是 250 X 232236 的字典，輸出是 3971, 250 格式的一篇文章的 word_embedding 格式。
RNN 層設定爲壓縮五倍。Dense 層只是中規中矩，輸出 10 維度的類別 vector。因爲是多類別分類，所以使用 softmax 的loss function。
![nenuro network model](./network_model.png)

### 訓練部分
How do you improve your performanc
Experiment settings and results (1%)
    Ex. Epochs, training time, hyperparameters, etc.
    
訓練過程中，第一次訓練只訓練一次，結果發現分類都是一個類別，還以爲自己寫錯了，後來嘗試過10次以後發現，其實是訓練太少的原因。

正式訓練時，增加 epochs 到 50 次，修改 batch_size 到 100 次，做一次正式訓練，kaggle 結果顯示 0.12 左右。大概花費時間 3 個小時左右。

In [ ]:
validation_split=0.1, batch_size=100, epochs = 50, verbose = 1)

再後來增加訓練次數，epochs 增加繼續訓練到 150 次，並增大 batch_size 的值到 150， kaggle 結果有明顯提升到 0.13 。這證明增加訓練次數是有效果的。batch_size 大一些，會增加梯度下降的準確性，提升模型預測的性能。

In [ ]:
validation_split=0.1, batch_size=150, epochs = 150, verbose = 1)

epochs 增加繼續訓練到 250 次，kaggle 結果有明顯提升到 0.14 。這證明增加訓練次數是有效果的。

In [ ]:
validation_split=0.1, batch_size=150, epochs = 150, verbose = 1)

In [ ]:
接下來, 我想另外一邊訓練舊的模型和資料, 另外我想清洗一下無用的英文字串資料數據, 並多訓練一次 word_embedding 的 model 結果。